In [169]:
import csv
import psycopg2
import requests

In [170]:
import networkx as nx
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
%matplotlib inline

In [171]:
with open('/home/faculty/millbr02/pub/un-general-debates.csv','r',encoding='latin-1') as f:
    csv_iter = csv.reader(f)
    db = []
    for row in csv_iter:
        db.append(row)

db[:5]

[['session', 'year', 'country', 'text'],
 ['44',
  '1989',
  'MDV',
  "ï»¿It is indeed a pleasure for me and the members of my delegation to extend to Ambassador Garba our sincere congratulations on his election to the presidency of the forty-fourth session of the General Assembly. His election to this high office is a well-deserved tribute to his personal qualities and experience. I am fully confident that under his able and wise leadership the Assembly will further consolidate the gains achieved during the past year.\nMy delegation associates itself with previous speakers in expressing its appreciation of the dedicated efforts of his predecessor, His Excellency Mr. Dante Caputo, for the exemplary manner in which he discharged his duties as President of the forty-third session of the General Assembly.\nAs in previous years, my delegation wishes to note its satisfaction with and gratitude for the assiduous and unrelenting efforts exerted by the Secretary-General of the United Nations i

In [172]:
conn = psycopg2.connect("postgresql://millbr02:@localhost/world")
cur = conn.cursor()
cur.execute("select name from country")
res = cur.fetchall()
matrix1 = {}
matrix2 = {}
for row in res:
    matrix1[row[0]] = 0
matrix1['Czechoslovakia'] = 0
matrix1['East Germany'] = 0
matrix1['European Union'] = 0
matrix1['Montenegro'] = 0
matrix1['Romania'] = 0
matrix1['South Sudan'] = 0
matrix1['East Timor'] = 0
matrix1['Democratic Yemen'] = 0
for row in res:
    copiedmatrix = matrix1.copy()
    matrix2[row[0]] = copiedmatrix
matrix2['Czechoslovakia'] = copiedmatrix
matrix2['East Germany'] = copiedmatrix
matrix2['European Union'] = copiedmatrix
matrix2['Montenegro'] = copiedmatrix
matrix2['Romania'] = copiedmatrix
matrix2['South Sudan'] = copiedmatrix
matrix2['East Timor'] = copiedmatrix
matrix2['Democratic Yemen'] = copiedmatrix

In [173]:
g = nx.Graph()
g.add_nodes_from(matrix2)

In [174]:
res = requests.get('https://opendata.socrata.com/api/views/mnkm-8ram/rows.json?accessType=DOWNLOAD')

j = res.json()

latlong = {}

for i in j['data']:
    lat = i[12]
    long = i[13]
    latlong[i[10]] = (lat, long)

for country in latlong:
    print(country + ': Latitude = ' + latlong[country][0] + ' Longitude = ' + latlong[country][1])

PER: Latitude = -10 Longitude = -76
MOZ: Latitude = -18.25 Longitude = 35
MMR: Latitude = 22 Longitude = 98
TKL: Latitude = -9 Longitude = -172
VGB: Latitude = 18.5 Longitude = -64.5
SYC: Latitude = -4.5833 Longitude = 55.6667
KIR: Latitude = 1.4167 Longitude = 173
MHL: Latitude = 9 Longitude = 168
SHN: Latitude = -15.9333 Longitude = -5.7
MKD: Latitude = 41.8333 Longitude = 22
BVT: Latitude = -54.4333 Longitude = 3.4
SRB: Latitude = 44 Longitude = 21
HKG: Latitude = 22.25 Longitude = 114.1667
MAC: Latitude = 22.1667 Longitude = 113.55
ESP: Latitude = 40 Longitude = -4
PYF: Latitude = -15 Longitude = -140
SLV: Latitude = 13.8333 Longitude = -88.9167
HUN: Latitude = 47 Longitude = 20
CHE: Latitude = 47 Longitude = 8
FRO: Latitude = 62 Longitude = -7
TCA: Latitude = 21.75 Longitude = -71.5833
NAM: Latitude = -22 Longitude = 17
LSO: Latitude = -29.5 Longitude = 28.5
AND: Latitude = 42.5 Longitude = 1.6
NIC: Latitude = 13 Longitude = -85
NOR: Latitude = 62 Longitude = 10
MLT: Latitude = 35

In [175]:
# first just create an in-memory list that contains all of the csv data
with open('/home/faculty/millbr02/pub/un-general-debates.csv','r',encoding='latin-1') as f:
    csv_iter = csv.reader(f)
    db = []
    for row in csv_iter:
        db.append(row)

In [179]:
unknown_country = []  
other_countries = {'CSK': ['Czechoslovakia', 'Europe', 'Eastern Europe'] , 
    'DDR': ['East Germany', 'Europe', 'Western Europe'],
    'EU': ['European Union', 'Europe','Western Europe'],
    'MNE': ['Montenegro', 'Europe', 'Eastern Europe'], 
    'ROU': ['Romania', 'Europe','Eastern Europe'], 
    'SSD': ['South Sudan', 'Africa', 'Northern Africa'], 
    'TLS': ['East Timor','Asia','Southeast Asia'], 
    'YDYE': ['Democratic Yemen', 'Asia', 'Middle East']}

for row in db[1:]:   # skip the first row
    cur.execute("""select code, name, region, continent 
                   from country where code = %s""", [row[2]])
    country = cur.fetchone()
    if country:
        row.append(country[1])  # name
        row.append(country[2])  # region
        row.append(country[3])  # continent
        if row[2] in latlong:
            row.append(latlong[row[2]][0])
            row.append(latlong[row[2]][1])
        else:
            row.append('Unknown Latitude')
            row.append('Unknown Longitude')
    else:
        unknown_country.append(row[2])
        row.append(other_countries[row[2]][0])  # name
        row.append(other_countries[row[2]][2])  # region
        row.append(other_countries[row[2]][1])  # continent
        if row[2] in latlong:
            row.append(latlong[row[2]][0])
            row.append(latlong[row[2]][1])
        else:
            row.append('Unknown Latitude')
            row.append('Unknown Longitude')
        
                
    quote = row[3]
    talkingcountry = row[4]
    for i in matrix2:
        matrix2[talkingcountry][i] += quote.count(i)
        if quote.count(i) > 20:
            g.add_edge(talkingcountry, i)


for i in matrix2:
    print(i + ' mentions...')
    for nation in matrix2[i]:
        print(nation + ' ' + str(matrix2[i][nation]) + ' times')
    print()
    print()

Democratic Yemen mentions...
Democratic Yemen 86 times
Gambia 0 times
Brazil 22 times
Afghanistan 188 times
France 34 times
Egypt 56 times
Portugal 42 times
Paraguay 0 times
Russian Federation 8 times
Saudi Arabia 4 times
Nicaragua 76 times
Cape Verde 8 times
Belgium 12 times
Ghana 0 times
Wallis and Futuna 0 times
Saint Helena 0 times
Guadeloupe 0 times
Congo, The Democratic Republic of the 0 times
Trinidad and Tobago 0 times
Greece 18 times
Seychelles 10 times
Syria 198 times
Thailand 10 times
Brunei 8 times
Burkina Faso 0 times
Faroe Islands 0 times
Tokelau 0 times
Austria 24 times
Bermuda 0 times
Lithuania 4 times
Ukraine 50 times
Fiji Islands 0 times
Puerto Rico 24 times
Haiti 12 times
Colombia 8 times
Belize 6 times
Mauritius 0 times
Poland 22 times
Barbados 0 times
Monaco 0 times
Nauru 2 times
Algeria 28 times
Comoros 2 times
Togo 0 times
Malaysia 16 times
European Union 156 times
Palau 0 times
Gibraltar 0 times
Tunisia 12 times
Tanzania 12 times
Iceland 6 times
Djibouti 12 time

Mexico 0 times
Ireland 2 times
Sierra Leone 2 times
Slovenia 0 times
Chile 2 times
Rwanda 6 times
Azerbaijan 0 times
Philippines 0 times
East Timor 296 times
Saint Pierre and Miquelon 0 times
Costa Rica 4 times
Guam 0 times
American Samoa 0 times
Sudan 10 times
Luxembourg 10 times
Italy 0 times
Turks and Caicos Islands 0 times
Equatorial Guinea 0 times
Morocco 0 times
Singapore 0 times
Kiribati 0 times
Falkland Islands 0 times
Congo 10 times
Cameroon 0 times
New Caledonia 0 times
Martinique 0 times
Kenya 2 times
Czech Republic 2 times
Zimbabwe 28 times
Virgin Islands, British 0 times
Solomon Islands 2 times
Aruba 0 times
Belarus 0 times
Guinea 128 times
United States 48 times
Guyana 0 times
Christmas Island 0 times
Papua New Guinea 0 times
Côte dIvoire 0 times
Mayotte 0 times
Cyprus 8 times
Jordan 4 times
Libyan Arab Jamahiriya 0 times
Nepal 0 times
Chad 10 times
Suriname 0 times
Montenegro 0 times
Croatia 0 times
Svalbard and Jan Mayen 0 times
Argentina 4 times
Malawi 0 times
Bulgari

Bahrain 0 times
Cambodia 0 times
Antigua and Barbuda 0 times
Swaziland 0 times
Saint Lucia 0 times
Guatemala 0 times
Romania 0 times
South Africa 0 times
Switzerland 0 times
Qatar 0 times
Turkmenistan 0 times
United Arab Emirates 0 times
Myanmar 0 times
Kuwait 0 times
Netherlands Antilles 0 times
Turkey 0 times
Mozambique 0 times
Senegal 0 times
Bhutan 0 times
Norfolk Island 0 times
Northern Mariana Islands 0 times
Honduras 0 times
San Marino 0 times
Slovakia 0 times
Bangladesh 0 times
Macao 0 times
Ecuador 0 times
Holy See (Vatican City State) 0 times
Tuvalu 0 times
Panama 0 times
Grenada 0 times
Pitcairn 0 times
North Korea 0 times
Norway 0 times
Maldives 0 times
India 0 times
China 0 times
Mali 0 times
Burundi 0 times
Taiwan 0 times
Lebanon 0 times
Lesotho 0 times
Uzbekistan 0 times
Marshall Islands 0 times
Cuba 0 times
Saint Vincent and the Grenadines 0 times
French Guiana 0 times
Finland 0 times
Iraq 0 times
Sweden 0 times
Cocos (Keeling) Islands 0 times
Germany 0 times
French Pol

Portugal 4 times
Paraguay 0 times
Russian Federation 0 times
Saudi Arabia 2 times
Nicaragua 4 times
Cape Verde 4 times
Belgium 2 times
Ghana 2 times
Wallis and Futuna 0 times
Saint Helena 0 times
Guadeloupe 0 times
Congo, The Democratic Republic of the 0 times
Trinidad and Tobago 0 times
Greece 0 times
Seychelles 2 times
Syria 8 times
Thailand 80 times
Brunei 8 times
Burkina Faso 0 times
Faroe Islands 0 times
Tokelau 0 times
Austria 4 times
Bermuda 0 times
Lithuania 2 times
Ukraine 0 times
Fiji Islands 0 times
Puerto Rico 2 times
Haiti 4 times
Colombia 4 times
Belize 2 times
Mauritius 4 times
Poland 4 times
Barbados 0 times
Monaco 2 times
Nauru 2 times
Algeria 6 times
Comoros 4 times
Togo 0 times
Malaysia 14 times
European Union 2 times
Palau 0 times
Gibraltar 0 times
Tunisia 2 times
Tanzania 4 times
Iceland 0 times
Djibouti 2 times
Liberia 0 times
United States Minor Outlying Islands 0 times
Oman 0 times
Sri Lanka 4 times
South Korea 38 times
Mauritania 0 times
Antarctica 0 times
Baha

Montenegro 0 times
Croatia 0 times
Svalbard and Jan Mayen 0 times
Argentina 0 times
Malawi 0 times
Bulgaria 0 times
East Germany 0 times
Angola 0 times
Kyrgyzstan 0 times
Virgin Islands, U.S. 0 times
Bouvet Island 0 times
Greenland 0 times
Macedonia 0 times
Yugoslavia 0 times
Indonesia 6 times
Albania 0 times
Yemen 4 times
Eritrea 0 times
Guinea-Bissau 0 times
Benin 0 times
Canada 2 times
Réunion 0 times
French Southern territories 0 times
Jamaica 0 times
Gabon 0 times
Anguilla 0 times
Zambia 0 times
Mongolia 0 times
Pakistan 14 times
Andorra 0 times
Iran 16 times
Georgia 0 times
Sao Tome and Principe 0 times
Israel 36 times
Cook Islands 0 times
Japan 4 times
Bosnia and Herzegovina 6 times
British Indian Ocean Territory 0 times
Armenia 0 times
Bolivia 0 times
Western Sahara 2 times
Vanuatu 0 times
Peru 0 times
Madagascar 0 times
Tonga 2 times
Czechoslovakia 0 times
Somalia 6 times
Botswana 0 times
South Sudan 2 times
Vietnam 6 times
Spain 2 times
Venezuela 0 times
El Salvador 0 times
N

...

Cocos (Keeling) Islands 0 times
Germany 0 times
French Polynesia 0 times
Estonia 0 times
Kazakstan 0 times
Central African Republic 0 times
Palestine 0 times
Niue 0 times
New Zealand 0 times
Cayman Islands 0 times
Hungary 0 times


Macedonia mentions...
Democratic Yemen 0 times
Gambia 0 times
Brazil 0 times
Afghanistan 8 times
France 0 times
Egypt 0 times
Portugal 0 times
Paraguay 0 times
Russian Federation 0 times
Saudi Arabia 0 times
Nicaragua 2 times
Cape Verde 0 times
Belgium 0 times
Ghana 0 times
Wallis and Futuna 0 times
Saint Helena 0 times
Guadeloupe 0 times
Congo, The Democratic Republic of the 0 times
Trinidad and Tobago 0 times
Greece 62 times
Seychelles 0 times
Syria 18 times
Thailand 0 times
Brunei 0 times
Burkina Faso 0 times
Faroe Islands 0 times
Tokelau 0 times
Austria 0 times
Bermuda 0 times
Lithuania 0 times
Ukraine 8 times
Fiji Islands 0 times
Puerto Rico 0 times
Haiti 0 times
Colombia 0 times
Belize 0 times
Mauritius 0 times
Poland 0 times
Barbados 0 times
Monaco 0 

Gabon 0 times
Anguilla 0 times
Zambia 0 times
Mongolia 0 times
Pakistan 12 times
Andorra 2 times
Iran 26 times
Georgia 0 times
Sao Tome and Principe 2 times
Israel 104 times
Cook Islands 0 times
Japan 6 times
Bosnia and Herzegovina 8 times
British Indian Ocean Territory 0 times
Armenia 0 times
Bolivia 0 times
Western Sahara 2 times
Vanuatu 0 times
Peru 0 times
Madagascar 0 times
Tonga 2 times
Czechoslovakia 0 times
Somalia 18 times
Botswana 0 times
South Sudan 20 times
Vietnam 4 times
Spain 2 times
Venezuela 0 times
El Salvador 0 times
Namibia 68 times
Heard Island and McDonald Islands 0 times
United Kingdom 2 times
Dominica 0 times
Bahrain 0 times
Cambodia 8 times
Antigua and Barbuda 0 times
Swaziland 0 times
Saint Lucia 0 times
Guatemala 22 times
Romania 2 times
South Africa 182 times
Switzerland 0 times
Qatar 0 times
Turkmenistan 0 times
United Arab Emirates 0 times
Myanmar 14 times
Kuwait 14 times
Netherlands Antilles 0 times
Turkey 2 times
Mozambique 6 times
Senegal 0 times
Bhutan

Antarctica 0 times
Bahamas 0 times
Nigeria 0 times
Niger 0 times
Montserrat 0 times
Liechtenstein 0 times
Tajikistan 40 times
Hong Kong 0 times
Malta 0 times
Saint Kitts and Nevis 0 times
Australia 0 times
Netherlands 0 times
Uruguay 2 times
Latvia 0 times
Denmark 0 times
Dominican Republic 0 times
Moldova 0 times
Micronesia, Federated States of 0 times
Uganda 0 times
South Georgia and the South Sandwich Islands 0 times
Laos 0 times
Samoa 0 times
Ethiopia 0 times
Mexico 0 times
Ireland 0 times
Sierra Leone 0 times
Slovenia 0 times
Chile 0 times
Rwanda 0 times
Azerbaijan 0 times
Philippines 0 times
East Timor 0 times
Saint Pierre and Miquelon 0 times
Costa Rica 0 times
Guam 0 times
American Samoa 0 times
Sudan 0 times
Luxembourg 0 times
Italy 2 times
Turks and Caicos Islands 0 times
Equatorial Guinea 0 times
Morocco 0 times
Singapore 0 times
Kiribati 0 times
Falkland Islands 0 times
Congo 0 times
Cameroon 0 times
New Caledonia 0 times
Martinique 0 times
Kenya 0 times
Czech Republic 0 ti

French Guiana 0 times
Finland 0 times
Iraq 26 times
Sweden 0 times
Cocos (Keeling) Islands 0 times
Germany 0 times
French Polynesia 0 times
Estonia 0 times
Kazakstan 0 times
Central African Republic 0 times
Palestine 4 times
Niue 0 times
New Zealand 0 times
Cayman Islands 0 times
Hungary 0 times


Germany mentions...
Democratic Yemen 0 times
Gambia 0 times
Brazil 16 times
Afghanistan 162 times
France 32 times
Egypt 8 times
Portugal 12 times
Paraguay 0 times
Russian Federation 2 times
Saudi Arabia 4 times
Nicaragua 2 times
Cape Verde 2 times
Belgium 0 times
Ghana 0 times
Wallis and Futuna 0 times
Saint Helena 0 times
Guadeloupe 0 times
Congo, The Democratic Republic of the 0 times
Trinidad and Tobago 0 times
Greece 4 times
Seychelles 2 times
Syria 82 times
Thailand 2 times
Brunei 4 times
Burkina Faso 0 times
Faroe Islands 0 times
Tokelau 0 times
Austria 0 times
Bermuda 0 times
Lithuania 2 times
Ukraine 24 times
Fiji Islands 0 times
Puerto Rico 0 times
Haiti 8 times
Colombia 0 times
Beli

Kazakstan 0 times
Central African Republic 0 times
Palestine 0 times
Niue 0 times
New Zealand 0 times
Cayman Islands 0 times
Hungary 0 times


Hungary mentions...
Democratic Yemen 0 times
Gambia 0 times
Brazil 0 times
Afghanistan 98 times
France 4 times
Egypt 12 times
Portugal 8 times
Paraguay 0 times
Russian Federation 6 times
Saudi Arabia 0 times
Nicaragua 12 times
Cape Verde 2 times
Belgium 0 times
Ghana 0 times
Wallis and Futuna 0 times
Saint Helena 0 times
Guadeloupe 0 times
Congo, The Democratic Republic of the 0 times
Trinidad and Tobago 0 times
Greece 4 times
Seychelles 2 times
Syria 26 times
Thailand 0 times
Brunei 2 times
Burkina Faso 0 times
Faroe Islands 0 times
Tokelau 0 times
Austria 16 times
Bermuda 0 times
Lithuania 2 times
Ukraine 8 times
Fiji Islands 0 times
Puerto Rico 0 times
Haiti 2 times
Colombia 2 times
Belize 2 times
Mauritius 0 times
Poland 12 times
Barbados 0 times
Monaco 2 times
Nauru 0 times
Algeria 4 times
Comoros 0 times
Togo 0 times
Malaysia 2 times
Europ

In [180]:
newmatrix = matrix1.copy()
for country in matrix2:
    for nation in matrix2[country]:
        newmatrix[nation] += matrix2[country][nation]

In [181]:
db2 = []
for i in db[1:]:
    db2.append([i[2],i[4],newmatrix[i[4]],i[7],i[8]])
print(db2)

[['MDV', 'Maldives', 970, '3.25', '73'], ['FIN', 'Finland', 1246, '64', '26'], ['NER', 'Niger', 3602, '16', '8'], ['URY', 'Uruguay', 2524, '-33', '-56'], ['ZWE', 'Zimbabwe', 5626, '-20', '30'], ['PHL', 'Philippines', 1162, '13', '122'], ['SDN', 'Sudan', 7632, '15', '30'], ['RUS', 'Russian Federation', 714, '60', '100'], ['CHN', 'China', 10570, '35', '105'], ['ESP', 'Spain', 2906, '40', '-4'], ['SUR', 'Suriname', 1014, '4', '-56'], ['ARG', 'Argentina', 2466, '-34', '-64'], ['SLV', 'El Salvador', 2362, '13.8333', '-88.9167'], ['MYS', 'Malaysia', 1694, '2.5', '112.5'], ['NPL', 'Nepal', 1822, '28', '84'], ['PRT', 'Portugal', 3878, '39.5', '-8'], ['COL', 'Colombia', 2382, '4', '-72'], ['BLR', 'Belarus', 1038, '53', '28'], ['MAR', 'Morocco', 2986, '32', '-5'], ['LCA', 'Saint Lucia', 1432, '13.8833', '-61.1333'], ['EGY', 'Egypt', 5160, '27', '30'], ['MEX', 'Mexico', 3568, '23', '-102'], ['BEL', 'Belgium', 1210, '50.8333', '4'], ['BRN', 'Brunei', 624, '4.5', '114.6667'], ['RWA', 'Rwanda', 3910

In [ ]:
labels = {}
gfig, gax = plt.subplots(figsize=(15,15))
for i in matrix2:
    labels[i] = i
#nx.draw_networkx_labels(g, nx.spring_layout(g,k=0.35), labels, font_size=10) (THIS WAS NOT A GOOD LINE)
nx.draw(g, nx.spring_layout(g,k=0.35), with_labels=True, node_size=10,ax=gax)

In [ ]:
'''gfig, gax = plt.subplots(figsize=(15,15))
hego = nx.ego(allg,'United States',radius=2)
pos = nx.spring_layout(hego, k=20*1/math.sqrt(len(mat.index)),scale=8.0)
nx.draw(hego, pos, ax=gax, node_size=1000, with_label=True, font_size=18)'''

In [ ]:
'''years = {}
for row in db[1:]:
    yr = row[1]
    country = row[2]
    if country not in unknown_country:
        region = row[5]
        if yr not in years:
            years[yr] = {}
        if region not in years[yr]:
            years[yr][region] = 1
        else:
            years[yr][region] += 1'''

In [ ]:
'''for yr in sorted(years):
    for region in sorted(years[yr]):
        print(yr, region, years[yr][region])'''